# UNSW-NB15

ALL CODE IS MINE UNLESS OTHERWISE STATED

In [ ]:
import os
import urllib.request
import zipfile
import pandas as pd
import numpy as np
import joblib
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix, classification_report

import tensorflow as tf
import keras_tuner as kt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dense, Dropout, BatchNormalization, Input, Add, GlobalAveragePooling1D, ReLU
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

In [2]:
url = "https://github.com/AdamWheatman/AI-IDS-Project/raw/refs/heads/main/Code%20Data/Training%20and%20Testing%20Sets.zip"
zip_filename = "UNSW-NB15.zip"

urllib.request.urlretrieve(url, zip_filename)
extract_folder = "data"
os.makedirs(extract_folder, exist_ok=True)

with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)

os.remove(zip_filename)

In [3]:
df = pd.read_csv('data/Training and Testing Sets/UNSW_NB15_training-set.csv')

In [4]:
label_encoders = {}

for col in ["proto", "service", "state", "attack_cat"]:
    df[col] = df[col].astype(str).fillna("unknown")
    label_encoders[col] = LabelEncoder()
    df[col] = label_encoders[col].fit_transform(df[col])

joblib.dump(label_encoders, 'label_encoders.pkl')

['label_encoders.pkl']

In [5]:
X = df.drop(columns=['id', 'proto', 'service', 'state', 'attack_cat', 'label'])
y = df['label']

In [6]:
feature_names = X.columns.tolist()
joblib.dump(feature_names, 'features.pkl')

['features.pkl']

In [7]:
# Standardize features
scaler = StandardScaler()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

joblib.dump(scaler, 'scaler.pkl')

['scaler.pkl']

In [8]:
# Convert Labels to Categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [9]:
# Compute class weights for balancing
class_weights = compute_class_weight('balanced', classes=np.unique(np.argmax(y_train, axis=1)), y=np.argmax(y_train, axis=1))
class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}

In [10]:
# ---- Reshape Data for CNN (Convert to 3D) ----
X_train_reshaped = X_train_scaled.reshape((X_train_scaled.shape[0], X_train_scaled.shape[1], 1))
X_test_reshaped = X_test_scaled.reshape((X_test_scaled.shape[0], X_test_scaled.shape[1], 1))

In [11]:
def residual_block(x, filters):
    shortcut = x
    # Apply Convolution, Batch Normalization, and ReLU
    x = Conv1D(filters, kernel_size=3, padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = Conv1D(filters, kernel_size=3, padding='same')(x)
    x = BatchNormalization()(x)

    # Adjust shortcut to match the shape if needed
    if shortcut.shape[-1] != filters:
        shortcut = Conv1D(filters, kernel_size=1, padding='same')(shortcut)

    # Add residual connection
    x = Add()([x, shortcut])
    x = ReLU()(x)

    return x


In [12]:
# ---- CNN Model with Hyperparameter Tuning ----
def build_cnn_model(hp):
    inputs = Input(shape=(X_train_reshaped.shape[1], 1))
    x = Conv1D(hp.Int('filters_1', 64, 256, step=64), kernel_size=3, activation='relu')(inputs)
    x = BatchNormalization()(x)
    x = MaxPooling1D(pool_size=2)(x)

    for i in range(hp.Int('num_res_blocks', 1, 3)):
        x = residual_block(x, filters=hp.Int(f'filters_res_{i}', 64, 256, step=64))

    x = GlobalAveragePooling1D()(x)
    x = Dense(hp.Int('dense_units', 64, 256, step=64), activation='relu')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(2, activation='softmax')(x)

    model = Model(inputs, outputs)
    model.compile(optimizer=Adam(learning_rate=hp.Choice('learning_rate', values=[0.001, 0.0001])),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model


In [13]:
# Compile the model
print("Starting Hyperparameter Tuning...")
tuner = kt.Hyperband(build_cnn_model, objective='val_accuracy', max_epochs=30, directory='cnn_tuner', project_name='Enhanced_CNN_IDS')
tuner.search(X_train_reshaped, y_train, epochs=10, validation_split=0.2)

Starting Hyperparameter Tuning...
Reloading Tuner from cnn_tuner\Enhanced_CNN_IDS\tuner0.json


In [14]:
# Get Best Model
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
best_cnn_model = tuner.hypermodel.build(best_hps)
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [15]:
best_cnn_model.fit(X_train_reshaped, y_train, epochs=30, batch_size=64, validation_split=0.2, class_weight=class_weight_dict, callbacks=[early_stop])

Epoch 1/30
1754/1754 ━━━━━━━━━━━━━━━━━━━━ 46s 24ms/step - accuracy: 0.9072 - loss: 0.2185 - val_accuracy: 0.9251 - val_loss: 0.1454
Epoch 2/30
1754/1754 ━━━━━━━━━━━━━━━━━━━━ 45s 25ms/step - accuracy: 0.9309 - loss: 0.1441 - val_accuracy: 0.9398 - val_loss: 0.1271
Epoch 3/30
1754/1754 ━━━━━━━━━━━━━━━━━━━━ 42s 24ms/step - accuracy: 0.9325 - loss: 0.1374 - val_accuracy: 0.9374 - val_loss: 0.1226
Epoch 4/30
1754/1754 ━━━━━━━━━━━━━━━━━━━━ 43s 25ms/step - accuracy: 0.9353 - loss: 0.1333 - val_accuracy: 0.9383 - val_loss: 0.1216
Epoch 5/30
1754/1754 ━━━━━━━━━━━━━━━━━━━━ 43s 24ms/step - accuracy: 0.9352 - loss: 0.1296 - val_accuracy: 0.9443 - val_loss: 0.1121
Epoch 6/30
1754/1754 ━━━━━━━━━━━━━━━━━━━━ 40s 23ms/step - accuracy: 0.9390 - loss: 0.1246 - val_accuracy: 0.9439 - val_loss: 0.1157
Epoch 7/30
1754/1754 ━━━━━━━━━━━━━━━━━━━━ 39s 22ms/step - accuracy: 0.9376 - loss: 0.1243 - val_accuracy: 0.9390 - val_loss: 0.1240
Epoch 8/30
1754/1754 ━━━━━━━━━━━━━━━━━━━━ 40s 23ms/step - accuracy: 0.9403 -

In [16]:
# Evaluate
loss, accuracy = best_cnn_model.evaluate(X_test_reshaped, y_test)
print(f'Test Accuracy: {accuracy:.4f}')

1096/1096 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.9415 - loss: 0.1165
Test Accuracy: 0.9421


In [17]:
# Train model
y_pred = best_cnn_model.predict(X_test_reshaped)

# Convert model probabilities to class labels
y_pred_labels = np.argmax(y_pred, axis=1)
y_test_labels = np.argmax(y_test, axis=1)

# False Negative: Attack (1) misclassified as Benign (0)
false_negatives = (y_pred_labels == 0) & (y_test_labels == 1)

# False Positive: Benign (0) misclassified as Attack (1)
false_positives = (y_pred_labels == 1) & (y_test_labels == 0)

# Print summary
print(f"False Negatives (Attacks misclassified as Benign): {sum(false_negatives)}")
print(f"False Positives (Benign misclassified as Attacks): {sum(false_positives)}")

# Show a few misclassified samples
misclassified_fn = np.where(false_negatives)[0][:5]  # First 5 false negatives
misclassified_fp = np.where(false_positives)[0][:5]  # First 5 false positives

print("\nFalse Negatives (Missed Attacks) - Example Indices:", misclassified_fn)
print("False Positives (Incorrectly Flagged Benign) - Example Indices:", misclassified_fp)

print(confusion_matrix(y_test_labels, y_pred_labels))
print(classification_report(y_test_labels, y_pred_labels))

1096/1096 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step
False Negatives (Attacks misclassified as Benign): 1138
False Positives (Benign misclassified as Attacks): 892

False Negatives (Missed Attacks) - Example Indices: [ 7 13 21 54 87]
False Positives (Incorrectly Flagged Benign) - Example Indices: [ 44  94 107 148 162]
[[10277   892]
 [ 1138 22762]]
              precision    recall  f1-score   support

           0       0.90      0.92      0.91     11169
           1       0.96      0.95      0.96     23900

    accuracy                           0.94     35069
   macro avg       0.93      0.94      0.93     35069
weighted avg       0.94      0.94      0.94     35069



In [18]:
# Evaluate the model
best_cnn_model.save('model.h5')
best_cnn_model.save('model.keras')